# Enkel OpenAI-bot i jupyter med kontext och konversationshistorik



##### Installera nödvändiga bibliotek om de inte redan är installerade (pip install openai ipywidgets)
### Importera nödvändiga bibliotek

In [6]:

from openai import OpenAI
import os
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import time


### Initiera OpenAI-klienten

In [7]:
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))  # Use environment variable for API key
# Alternatively, set your API key directly (not recommended for security):
# client = OpenAI(api_key="sk-your-actual-key-here")


 ### Dokumentladdning med återkoppling

In [8]:
def load_document(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            display(Markdown("✅ Document loaded successfully!"))
            return content
    except FileNotFoundError:
        display(Markdown("❌ **Error:** File not found. Please check the path."))
        return None

### Chattgränssnitt
#####
##### Ge Botten en roll, i detta fall som en assistent:
- På denna rad: `"role": "system", "content": "Du är en hjälpsam assistent. Använd följande innehåll som referens: "`

##### Chattens historik läggs till på detta sätt:
- Lägga till användarens meddelanden med `{"role": "user", "content": user_input}`
- Lägga till assistentens svar med `{"role": "assistant", "content": bot_response}`
- Använda denna historik i meddelandets kontext: `messages = [...] + self.history + [...]`


In [9]:

class ChatInterface:
    def __init__(self, document_content):
        self.document_content = document_content
        self.history = []
        self.setup_widgets()
        
    def setup_widgets(self):
        self.input_box = widgets.Text(
            placeholder='Skriv din fråga här (eller "rensa" för att återställa chatten)...',
            description='Du:',
            layout=widgets.Layout(width='70%'),
            continuous_update=False  
        )
        
        self.submit_button = widgets.Button(
            description='Send',
            button_style='primary',
            layout=widgets.Layout(width='20%')
        )
        
        self.input_area = widgets.HBox([self.input_box, self.submit_button])
        
        self.output_box = widgets.Output(
            layout=widgets.Layout(
                border='1px solid #ddd',
                padding='10px',
                margin='10px 0',
                max_height='400px',
                overflow_y='auto'
            )
        )
        self.status = widgets.HTML(value="<i>Ready...</i>")
        
    def handle_submit(self, _):
        self.process_input(self.input_box.value)
        
    def on_submit(self, change):
        if change["name"] == "value" and change["type"] == "change" and change.get("new", "").endswith("\n"):
            self.input_box.value = self.input_box.value.rstrip()
            self.process_input(self.input_box.value)
            
    def process_input(self, user_input):
        if not user_input.strip(): 
            return
            
        self.input_box.value = ''
        
        if user_input.lower() == "clear":
            with self.output_box:
                clear_output()
                self.history = []
            return
                
        with self.output_box:
            print(f"You: {user_input}")
            self.status.value = "<i>Tänker...</i>"
            
            try:
                messages = [
                    {"role": "system", "content": "Du är en hjälpsam assistent. Använd följande innehåll som referens: " + self.document_content},
                ] + self.history + [{"role": "user", "content": user_input}]
                
                response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=messages
                )
                
                bot_response = response.choices[0].message.content
                print(f"Assistant: {bot_response}\n{'─' * 50}\n")
                
                self.history.append({"role": "user", "content": user_input})
                self.history.append({"role": "assistant", "content": bot_response})
                
            except Exception as e:
                print(f"❌ Error: {str(e)}")
            
            finally:
                self.status.value = "<i>Ready...</i>"

### Initiera och visa gränssnitt
##### Lägg till dokument pathhway här

In [10]:

document_content = load_document("faq_hm.txt")
if document_content:
    chat = ChatInterface(document_content)
    display(Markdown("### Interaktiv assistent"))
    display(chat.input_area, chat.status, chat.output_box)
    chat.input_box.observe(chat.on_submit, names="value")
    chat.submit_button.on_click(chat.handle_submit)

✅ Document loaded successfully!

### Interaktiv assistent

HTML(value='<i>Ready...</i>')

Output(layout=Layout(border_bottom='1px solid #ddd', border_left='1px solid #ddd', border_right='1px solid #dd…